In [33]:
tmp = !pwd
current_path = tmp[0]

import timeit

In [40]:
import psycopg2
from psycopg2 import Error

try:
    connection = psycopg2.connect(user = "kevin",
                                  password = "",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "flightly")

    cursor = connection.cursor()
    
    drop_all_tables = """
        DROP SCHEMA public CASCADE;
        CREATE SCHEMA public;
        """
    create_airlines_table = """
        CREATE TABLE airlines (IATA TEXT PRIMARY KEY NOT NULL,
          NAME TEXT);
        """
    create_airports_table = """
        CREATE TABLE airports (IATA TEXT PRIMARY KEY NOT NULL,
          NAME TEXT,
          CITY TEXT,
          STATE TEXT,
          COUNTRY TEXT,
          LATITUDE TEXT,
          LONGITUDE TEXT
          );
        """
    
    create_flights_table = """
        CREATE TABLE flights
        (
            year text,
            month text,
            day text,
            day_of_week text,
            airline text,
            flight_number text,
            tail_number text,
            origin_airport text,
            destination_airport text,
            scheduled_departure text,
            departure_time text,
            departure_delay text,
            taxi_out text,
            wheels_off text,
            scheduled_time text,
            elapsed_time text,
            air_time text,
            distance text,
            wheels_on text,
            taxi_in text,
            scheduled_arrival text,
            arrival_time text,
            arrival_delay text,
            diverted text,
            cancelled text,
            cancellation_reason text,
            air_system_delay text,
            security_delay text,
            airline_delay text,
            late_aircraft_delay text,
            weather_delay text
        )
        """
    
    load_airlines_csv = f"""
        COPY airlines
        FROM '{current_path}/csv/airlines.csv' DELIMITER ',' HEADER CSV;
        """
    load_airports_csv = f"""
        COPY airports
        FROM '{current_path}/csv/airports.csv' DELIMITER ',' HEADER CSV;
        """
    load_flights_csv = f"""
        COPY flights
        FROM '{current_path}/csv/flights_reduced.csv' DELIMITER ',' HEADER CSV;
        """
    
    cursor.execute(drop_all_tables)
    cursor.execute(create_airlines_table)
    cursor.execute(create_airports_table)
    cursor.execute(create_flights_table)
    
    cursor.execute(load_airlines_csv)
    cursor.execute(load_airports_csv)
    cursor.execute(load_flights_csv)
    
    def hello():
        cursor.execute("SELECT * from flights where origin_airport = 'ATL' and airline = 'AA'")
        return cursor.fetchall()
    
    print(timeit.timeit(hello, number=200))
    
    connection.commit()
    print("Table created successfully in PostgreSQL ")

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table", error)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

0.7851513749997139
Table created successfully in PostgreSQL 
PostgreSQL connection is closed
